In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import simpletransformers
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn
import torch
import re
import string
import torchvision
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
import optuna
from sklearn.metrics import accuracy_score
from simpletransformers.classification import ClassificationModel

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
df = pd.read_csv('/content/sample_data/Лист1_csv_final_psycho_final_psycho_Лист1_csv_csv (2).csv', header = 0)


In [ ]:
df.head()

,Text,Syndrome
0,"I hate psychiatry with a passion, I refuse to ...",Bipolar
1,I went through an existential crisis wherein I...,Bipolar
2,I can't talk to my mom or my dad about the tru...,Bipolar
3,I tried telling my doctor that some of the med...,Bipolar
4,Then for some reason I started to notice exces...,Bipolar


In [ ]:
df['Syndrome'] = df['Syndrome'].map({'Schizophrenia': 0, 'PTSD': 1, 'None': 2, 'Intellectual disability': 3, 'Depression': 4, 'Bipolar': 5, 'ADHD': 6}).values

In [ ]:
df = df.set_axis(['text', 'labels'], axis=1, inplace=False)
df

<ipython-input-22-2aaf86c0a264>:1: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df = df.set_axis(['text', 'labels'], axis=1, inplace=False)


,text,labels
0,"I hate psychiatry with a passion, I refuse to ...",5
1,I went through an existential crisis wherein I...,5
2,I can't talk to my mom or my dad about the tru...,5
3,I tried telling my doctor that some of the med...,5
4,Then for some reason I started to notice exces...,5
...,...,...
705,I had always been treated with antidepressant...,5
706,I couldn't sleep. At one time I got about 12 h...,5
707,I almost only get mixed states that come with ...,5
708,I've gone much of my life. Anywho...\nI had a...,5


In [ ]:
df.head()

,text,labels
0,"I hate psychiatry with a passion, I refuse to ...",5
1,I went through an existential crisis wherein I...,5
2,I can't talk to my mom or my dad about the tru...,5
3,I tried telling my doctor that some of the med...,5
4,Then for some reason I started to notice exces...,5


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['labels'], test_size=0.2, random_state=42)

train_df = pd.DataFrame(X_train)
train_df['labels'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['labels'] = y_test

train_df.shape, eval_df.shape

((568, 2), (142, 2))

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

#данный код содержит в себе проверку моделей simpletransformers а также оптимизатор optuna
def objective(trial):
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-3)
  weight_decay = trial.suggest_loguniform("weight_decay", 1e-8, 1e-3)

# Создаём модель (тут можно на выбор взять xlnet, bert или roberta)
  model = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=7, args={'num_train_epochs':15,
                                                                              'train_batch_size':16,
                                                                              'max_seq_length':256,
                                                                              'learning_rate': learning_rate,
                                                                              'weight_decay': weight_decay,
                                                                              'overwrite_output_dir': True},
                              use_cuda=False)

  # Обучаем её
  model.train_model(train_df)

  # Получаем предсказания
  predictions, raw_outputs = model.predict(X_test.tolist())

  #делаем точность для того чтобы оптимизировать модель по данному параметру
  accuracy = accuracy_score(y_test, predictions)

  return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)
print("Best hyperparametrs: {}".format(study.best_params))
print("Best hyperparametrs: {}".format(study.best_value))